In [1]:
import urllib .request
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import re
from datetime import datetime, timezone, timedelta

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 뉴스 제목으로부터 링크를 가져오는 함수 정의
def get_link_from_title(page_num, URL):
  for i in range(page_num):
    current_page_num = 1 + i
    position = URL.index('pageNum=')
    URL_with_page_num = URL[:position+5] + str(current_page_num) + URL[position+5:]
    # HTTP 요청 객체 생성 (크롤링 방지 우회용 User-Agent 설정)
    URL_with_page_num = urllib.request.Request(URL_with_page_num,
                                               headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})

    source_code_form_url = urllib.request.urlopen(URL_with_page_num)  # URL에 요청을 보내고 HTML 응답 수신
    soup = BeautifulSoup(source_code_form_url, 'html.parser', from_encoding='utf-8')  # BeautifulSoup으로 HTML 파싱

    for title in soup.select('div.list_search > ul.news_list div.text a'):
        TITLE_OF_ARTICLE.append(title.get_text())
        article_URL = title['href']   # <a> 태그의 href 속성값 (기사 링크) 추출
        URL_OF_ARTICLE.append(article_URL)   # 기사 링크를 리스트에 추가

    if (i) % 10 == 0:   # 매 10번째 루프마다
        time.sleep(2)   # 서버 과부하 방지를 위한 2초 대기
        print(f'{(i) * 10} articles complete')     # 진행 상황 출력

In [3]:
# 전체 기사 수를 받아 필요한 페이지 수를 계산하는 함수 정의
# 아래 코드는 내가 보고있는 페이지의 게시물 개수에 따라 숫자가 달라지므로 반드시 확인

def get_total_num_of_article(number):
    if(number % 20 >= 1):   # 20으로 나눈 나머지가 1 이상이면 (즉, 나누어떨어지지 않으면)
        page_num = int((number / 20) + 1)   # 몫에 1을 더하여 추가 페이지 확보 (반올림 효과)
    else:   # 20으로 나누어떨어지는 경우
        page_num = int(number / 20)     # 정확히 나눈 몫이 곧 페이지 수
    return page_num



In [5]:
from urllib.parse import urlencode, urlsplit, urlunsplit, parse_qsl

# 1) URL 템플릿을 깔끔하게 정의 (pageNum만 변수)
TARGET_URL_TPL = (
    "https://search.etnews.com/etnews/search.html"
    "?category=CATEGORY1"
    "&kwd=%ED%9C%B4%EB%A8%B8%EB%85%B8%EC%9D%B4%EB%93%9C"
    "&pageNum={page}"
    "&pageSize=20&reSrchFlag=false&sort=1"
    "&startDate=20240901&endDate=20250915"
    "&detailSearch=true"
    "&preKwd%5B0%5D=%ED%9C%B4%EB%A8%B8%EB%85%B8%EC%9D%B4%EB%93%9C"
    )

TITLE_OF_ARTICLE, URL_OF_ARTICLE = [], []

def get_link_from_title(page_num: int):
    for i in range(page_num):
        current_page_num = i + 1
        page_url = TARGET_URL_TPL.format(page=current_page_num)

        req = urllib.request.Request(
            page_url,
            headers={'User-Agent':
                     'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
        )
        with urllib.request.urlopen(req) as resp:
            soup = BeautifulSoup(resp, 'html.parser', from_encoding='utf-8')

        # 리스트 선택자 확인 (현재 선택자는 맞습니다)
        for a in soup.select('div.list_search > ul.news_list div.text a'):
            TITLE_OF_ARTICLE.append(a.get_text(strip=True))
            URL_OF_ARTICLE.append(a.get('href'))

        # 진행상황 로그 (10페이지마다)
        if (current_page_num % 10) == 0:
            print(f'{current_page_num} pages crawled')
            time.sleep(2)  # 서버 배려

# 사용
page_num = get_total_num_of_article(903)  # 20개/페이지 기준
TITLE_OF_ARTICLE, URL_OF_ARTICLE = [], []
get_link_from_title(page_num)

df = pd.DataFrame({'Url': URL_OF_ARTICLE, 'Title': TITLE_OF_ARTICLE})
print(len(df))
df


10 pages crawled
20 pages crawled
30 pages crawled
40 pages crawled
903


,Url,Title
0,https://www.etnews.com/20250915000266,100kg도 번쩍···값싸고 머리 좀 쓴다는 산업·가정용 로봇 등장
1,https://www.etnews.com/20250915000293,"HD현대삼호, AI 휴머노이드 로봇·물류 자동화 기술 개발로 미래 조선소 구축"
2,https://www.etnews.com/20250915000110,"두산로보틱스, R&D 통합 '이노베이션 센터' 설립"
3,https://www.etnews.com/20250915000015,"KIRO, '제25회 한국지능로봇경진대회' 개최…AI화가로봇, 오목로봇, 반려로봇 ..."
4,https://www.etnews.com/20250914000079,"[ET시선]AI 올인한 폭스바겐과 샤오펑, 그리고 현대차"
...,...,...
898,https://www.etnews.com/20240903000210,[ET라씨로] 샤페론 주가 上…“아토피 치료제 美 FDA 임상2상 안정성 검증”
899,https://www.etnews.com/20240902000402,"[경제 활성화, 지역에서 찾자]〈1〉홍준표 대구광역시장, “ABB 등 5대 신산업 ..."
900,https://www.etnews.com/20240902000240,"레인보우로보틱스, 한국천문연구원에 광학감시시스템 공급"
901,https://www.etnews.com/20240902000181,"[ET라씨로] 에코프로비엠, 주가 8%↑…코스닥 시총 1위 탈환"


In [7]:
# DataFrame을 CSV 파일로 저장
df.to_csv("/content/drive/MyDrive/NLP/robot_news/test(1).csv",     # 저장할 파일 경로 (Google Drive 내 지정 경로)
    index=False,    # 행 번호(인덱스)를 저장하지 않음
    encoding='utf-8-sig'    # 인코딩 방식: 한글 깨짐 방지를 위해 'utf-8-sig' 사용
)

In [9]:
import re, time, urllib.request
from bs4 import BeautifulSoup
from datetime import datetime

# -------- 본문/날짜 파서 --------
def parse_etnews_article(url: str, timeout=30):
    req = urllib.request.Request(
        url,
        headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
    )
    with urllib.request.urlopen(req, timeout=timeout) as resp:
        soup = BeautifulSoup(resp, "html.parser", from_encoding="utf-8")

    # 1) 본문 후보 선택자들 (상황별로 다름)
    body_selectors = [
        "div.article_body p",          # 일반 케이스
        "div#articleBody p",           # id로 잡히는 케이스
        "div.article_body",            # <p> 없이 텍스트가 바로 들어가는 케이스
        "article#NewsContent p",       # 기사 영역이 article 태그인 케이스
        "div#newsContent p",
    ]

    body_texts = []
    for sel in body_selectors:
        nodes = soup.select(sel)
        if nodes:
            # <p> 리스트일 때
            if hasattr(nodes[0], "get_text"):
                body_texts = [n.get_text(" ", strip=True) for n in nodes]
                # 너무 짧은 광고성/공백만 나오면 다음 셀렉터로 넘어감
                joined = " ".join(t for t in body_texts if t)
                if len(joined) >= 80:
                    body = joined
                    break
            # 단일 div에서 바로 텍스트 추출
            else:
                txt = " ".join([n.get_text(" ", strip=True) for n in nodes])
                if len(txt) >= 80:
                    body = txt
                    break
    else:
        # 모든 셀렉터가 실패하면 fall-back: 기사 영역 전체에서 p 태그 긁기
        nodes = soup.select("p")
        body = " ".join(n.get_text(" ", strip=True) for n in nodes)
        body = re.sub(r"\s+", " ", body).strip()

    # 2) 날짜 추출 (신뢰도 순서대로)
    date_txt = None

    # (a) header 영역의 time 태그
    t = soup.select_one("div.article_header .time time")
    if t and t.get("datetime"):
        date_txt = t["datetime"]
    elif t:
        date_txt = t.get_text(strip=True)

    # (b) og/article 메타태그
    if not date_txt:
        m = soup.select_one('meta[property="article:published_time"]') or \
            soup.select_one('meta[name="pubdate"]') or \
            soup.select_one('meta[name="date"]')
        if m and m.get("content"):
            date_txt = m["content"]

    # (c) 기타 흔한 위치
    if not date_txt:
        alt = soup.select_one("span.date") or soup.select_one("div.time") or soup.select_one("div.news_time")
        if alt:
            date_txt = alt.get_text(" ", strip=True)

    # 3) 날짜 정규화 (ISO-8601로)
    DATE = "NONE"
    if date_txt:
        # 2025-09-30 10:21, 2025.09.30 10:21, 2025-09-30T10:21 등 대응
        cleaned = date_txt.replace("입력", "").replace(":", ":", 1).strip()
        cleaned = cleaned.replace(".", "-").replace("T", " ")
        cleaned = re.sub(r"[^0-9:\-\s]", " ", cleaned)
        cleaned = re.sub(r"\s+", " ", cleaned).strip()
        # 뼈대만 추출: YYYY-MM-DD HH:MM(:SS)?
        m = re.search(r"(\d{4}-\d{2}-\d{2})(?:\s+(\d{2}:\d{2}(?::\d{2})?))?", cleaned)
        if m:
            day = m.group(1)
            hhmm = m.group(2) or "00:00:00"
            if len(hhmm) == 5:  # HH:MM → HH:MM:00
                hhmm = hhmm + ":00"
            DATE = f"{day} {hhmm}"

    # 본문 후처리(공백 정리)
    CONTENT = re.sub(r"\s+", " ", body).strip()
    return DATE, CONTENT


In [10]:
# 기사 URL 목록을 입력으로 받아 각 URL에서 텍스트를 추출하는 함수
def get_texts_from_news(URL):
    # URL 목록의 각 URL에 대하여 반복 실행
    for i in range(len(URL)):
        url = urllib.request.Request(URL[i], headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'})
        source_code_from_url = urllib.request.urlopen(url)
        soup = BeautifulSoup(source_code_from_url, 'html.parser', from_encoding='utf-8')

        content = soup.select('div.article_body p')

        try:
            if content == []:   # 만약 기본 구조로 본문이 없을 경우
                content = soup.select('p.content_text')     # 대체 구조로 본문 재시도

                CONTENT = []    # 본문 저장용 리스트 초기화
                for item in content:    # 각 문단 반복
                    CONTENT.append(item.get_text())     # 텍스트 추출 후 리스트에 추가

                CONTENT = " ".join(CONTENT)     # 하나의 문자열로 결합
                CONTENT_OF_ARTICLE.append(CONTENT)      # 전역 본문 리스트에 추가

            else:   # 본문이 정상적으로 추출된 경우
                CONTENT = []
                for item in content:
                    CONTENT.append(item.get_text())     # 동일한 방식으로 텍스트 추출

                CONTENT = " ".join(CONTENT)
                CONTENT_OF_ARTICLE.append(CONTENT)

        except:     # 본문 추출 실패 시
            CONTENT = 'NONE'
            CONTENT_OF_ARTICLE.append(CONTENT)

        try:
            date = soup.select_one('div.article_header div.time > time')      # 기본 구조에서 날짜 추출 시도
            if date == None:    # 없으면 대체 구조로 재시도
                date = soup.select_one('div.article_header div.time > time').get_text()
                DATE = date[3:13]   # 날짜 문자열 일부 추출
                DATE_OF_ARTICLE.append(DATE)
            else:
                DATE = date.get_text()[3:22]
                DATE_OF_ARTICLE.append(DATE)

        except:     # 날짜 추출 실패 시
            DATE = 'NONE'
            DATE_OF_ARTICLE.append(DATE)


        # 서버 부하 방지 및 진행 모니터링
        if i % 5 == 0:
            time.sleep(2)    # 5개마다 2초 대기
        if i % 100 == 0:    # 100개마다 진행 상태 출력
            print(f'{i} articles complete')

In [11]:
CONTENT_OF_ARTICLE = []
DATE_OF_ARTICLE = []

get_texts_from_news(df['Url'])

0 articles complete
100 articles complete
200 articles complete
300 articles complete
400 articles complete
500 articles complete
600 articles complete
700 articles complete
800 articles complete
900 articles complete


In [12]:
print(CONTENT_OF_ARTICLE)

["독일 휴머노이드 로봇 스타트업 뉴라 로보틱스(Neura Robotics)가 가정과 산업 현장에서 활용할 수 있는 차세대 로봇 '4NE1(포애니원)'을 선보였습니다.\n\n포애니원의 가장 큰 특징은 뛰어난 '인지 지능(Cognitive Intelligence)'입니다. 이를 통해 주변 환경을 스스로 인식하고 판단하며 학습할 수 있어 사람과 사물을 구별하고 상황에 맞춰 행동을 조절합니다.\n\n또한 '인공 피부' 센서가 탑재되어 실제 접촉이 일어나기 직전 터치를 감지합니다. 이 덕분에 안전 펜스 없이도 사람과 함께 작업할 수 있는 코봇(cobot) 형태로 활용됩니다. 1.8미터의 키에 최대 100kg까지 들어 올릴 수 있으며 24시간 연속 가동이 가능한 강력한 성능도 갖췄습니다.\n\n뉴라 로보틱스는 이 로봇을 소비자들이 감당할 수 있는 가격으로 출시할 예정이라고 밝혔습니다. 이는 기존의 산업용 로봇 시장을 넘어 일반 가정용 시장으로 진출하겠다는 회사의 큰 목표로 보여집니다.\n\n이창민 기자 re34521@etnews.com\n", "HD현대삼호가 지속가능한 조선산업 생태계 조성을 위해 인공지능(AI) 기반 휴머노이드 로봇과 물류 자동화 기술 개발을 통한 미래 조선소 구축에 속도를 낸다.\n\nHD현대삼호는 15일 판넬공장에서 김재을 HD현대삼호 대표, 현신균 LG CNS 대표, 김완수 HD현대로보틱스 대표를 비롯해 HD한국조선해양 등 관계자들이 참석한 가운데 '안전한 공정 운영 및 효율화를 위한 휴머노이드 및 물류자동화 기술 개발' 업무협약을 체결했다고 밝혔다.\n\n협약의 주요 내용은 △용접 외 측정·성형·관제 등 다양한 생산 활동에 적용 가능한 휴머노이드 개발 △자율이동로봇 개발을 통한 조선소 물류 자동화 시스템 구축 등이다.\n\n이를 위해 HD현대삼호는 제조 데이터 확보와 현장 인프라 조성을, LG CNS는 조선산업용 AI 및 데이터 융합 플랫폼 구축 및 운영을 맡고, HD현대로보틱스는 공정별 특화 AI 모션 제어 기술을 개발·제공한다. HD한국조선

In [13]:
df = pd.DataFrame(list(zip(df['Url'], df['Title'], CONTENT_OF_ARTICLE, DATE_OF_ARTICLE,)), columns =['url', 'title', 'content', 'published_at'])

print(len(df))
df.tail()

903


,url,title,content,published_at
898,https://www.etnews.com/20240903000210,[ET라씨로] 샤페론 주가 上…“아토피 치료제 美 FDA 임상2상 안정성 검증”,샤페론이 현재 개발 중인 차세대 아토피 치료제가 미국 식품의약국(FDA) 임상 2상...,: 2024-09-03 13:14
899,https://www.etnews.com/20240902000402,"[경제 활성화, 지역에서 찾자]〈1〉홍준표 대구광역시장, “ABB 등 5대 신산업 ...",대구는 섬유산업이 쇠락하면서 지난 30년간 경제 침체기를 겪어왔다. 1인당 지역내총...,: 2024-09-03 11:00
900,https://www.etnews.com/20240902000240,"레인보우로보틱스, 한국천문연구원에 광학감시시스템 공급",레인보우로보틱스는 한국천문연구원(천문연)과 78억원 규모 중·고궤도 광학감시시스템 ...,: 2024-09-02 13:52
901,https://www.etnews.com/20240902000181,"[ET라씨로] 에코프로비엠, 주가 8%↑…코스닥 시총 1위 탈환","2차전지 관련주가 오름세를 보이는 가운데, 에코프로비엠이 알테오젠을 제치고 코스닥시...",: 2024-09-02 11:53
902,https://www.etnews.com/20240902000064,"[ET라씨로] 압타머사이언스, 인슐린 질환 치료 물질 美 특허 등록…주가 上",압타머사이언스가 최근 미국특허청(USPTO)에 '인슐린 수용체 압타머와 이를 포함하...,: 2024-09-02 09:44


In [14]:
df['content'].replace('NONE', np.nan, inplace=True) # 'Content' 열에서 'NONE' 문자열을 NaN 값으로 대체
df['content'].replace('', np.nan, inplace=True) # 'Content' 열에서 빈 문자열을 NaN 값으로 대체
df.dropna(subset=['content'], inplace=True) # 'Content' 열에 NaN 값을 포함하는 모든 행을 삭제

# 처리 후 남은 행의 개수 출력
print(len(df))

903


/tmp/ipython-input-1412268364.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['content'].replace('NONE', np.nan, inplace=True) # 'Content' 열에서 'NONE' 문자열을 NaN 값으로 대체
/tmp/ipython-input-1412268364.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df

In [16]:
df.to_csv("/content/drive/MyDrive/NLP/robot_news/ETNews.csv",index=False, encoding='utf-8-sig')

In [17]:
# published_at 열을 ISO8601 포맷으로 변환
def convert_to_iso8601(date_str):
    """
    ': 2025-08-29 20:53' 형태의 문자열을 ISO8601 포맷으로 변환
    """
    try:
        # 앞뒤 공백 제거 후 ': ' 부분을 제거하고 날짜와 시간 추출
        clean_date = date_str.strip().replace(': ', '')
        # datetime 객체로 파싱
        dt = datetime.strptime(clean_date, '%Y-%m-%d %H:%M')
        # ISO8601 포맷으로 변환 (초 단위까지 포함)
        return dt.strftime('%Y-%m-%dT%H:%M:%S')
    except Exception as e:
        print(f"변환 오류: '{date_str}' -> {e}")
        return 'NONE'

# published_at 열에 변환 함수 적용
df['published_at_iso'] = df['published_at'].apply(convert_to_iso8601)

# 변환 결과 확인
print("\n변환 전:")
print(df['published_at'].head())
print("\n변환 후 (ISO8601):")
print(df['published_at_iso'].head())

# 디버깅: 실제 데이터 형태 확인
print("\n실제 데이터 형태 확인:")
for i in range(5):
    print(f"Index {i}: '{df['published_at'].iloc[i]}' (길이: {len(str(df['published_at'].iloc[i]))})")


변환 전:
0     : 2025-09-15 17:10
1     : 2025-09-15 14:20
2     : 2025-09-15 10:32
3     : 2025-09-15 08:47
4     : 2025-09-14 16:00
Name: published_at, dtype: object

변환 후 (ISO8601):
0    2025-09-15T17:10:00
1    2025-09-15T14:20:00
2    2025-09-15T10:32:00
3    2025-09-15T08:47:00
4    2025-09-14T16:00:00
Name: published_at_iso, dtype: object

실제 데이터 형태 확인:
Index 0: ' : 2025-09-15 17:10' (길이: 19)
Index 1: ' : 2025-09-15 14:20' (길이: 19)
Index 2: ' : 2025-09-15 10:32' (길이: 19)
Index 3: ' : 2025-09-15 08:47' (길이: 19)
Index 4: ' : 2025-09-14 16:00' (길이: 19)


In [18]:
df.drop('published_at', axis=1, inplace=True)

In [ ]:
df

,url,title,content,published_at_iso
0,https://www.etnews.com/20250915000266,100kg도 번쩍···값싸고 머리 좀 쓴다는 산업·가정용 로봇 등장,독일 휴머노이드 로봇 스타트업 뉴라 로보틱스(Neura Robotics)가 가정과 ...,2025-09-15T17:10:00
1,https://www.etnews.com/20250915000337,"마음AI, 온디바이스·자율제어 기반 국방 AI사업 시동","피지컬 인공지능(AI) 전문기업 마음AI가 온디바이스 AI와 전술 AI, 자율제어 ...",2025-09-15T14:59:00
2,https://www.etnews.com/20250915000293,"HD현대삼호, AI 휴머노이드 로봇·물류 자동화 기술 개발로 미래 조선소 구축",HD현대삼호가 지속가능한 조선산업 생태계 조성을 위해 인공지능(AI) 기반 휴머노이...,2025-09-15T14:20:00
3,https://www.etnews.com/20250915000292,"AI 특화 모델 개발, '대기업 참여 제한'으로 강소기업 각축전",정부가 새롭게 추진하는 독자 인공지능(AI) 특화 파운데이션 모델 개발 프로젝트에 ...,2025-09-15T14:20:00
4,https://www.etnews.com/20250915000144,"디캠프, 배치 4기 스타트업 10개사 최종 선발",디캠프(대표 박영훈)는 딥테크 스타트업 대상으로 운영되는 디캠프 배치 4기에 스타트...,2025-09-15T11:00:00
...,...,...,...,...
739,https://www.etnews.com/20240902000275,"현대차그룹, '월드 스마트시티 엑스포' 참가",현대차그룹은 3일부터 5일까지 '더 나은 삶이 여기에'라는 주제로 경기도 고양시 킨...,2024-09-02T14:17:00
740,https://www.etnews.com/20240902000240,"레인보우로보틱스, 한국천문연구원에 광학감시시스템 공급",레인보우로보틱스는 한국천문연구원(천문연)과 78억원 규모 중·고궤도 광학감시시스템 ...,2024-09-02T13:52:00
741,https://www.etnews.com/20240902000164,"모양 바꿔 바위·계단 손쉽게 넘는다…기계연, 실시간 휠 강성 변화 기술 '세계 최초...","바퀴 강성을 실시간 변화시켜 계단·바위 등 다양한 장애물을 극복할 수 있는 휠, 이...",2024-09-02T11:25:00
742,https://www.etnews.com/20240830000225,"혼란 속 두산, 원전 경쟁력 확보 집중…플랜B는 성공할까",두산그룹이 두산밥캣와 두산로보틱스의 합병안은 철회했지만 두산에너빌리티에서 두산밥켓을...,2024-09-01T15:37:00


In [19]:
df.to_csv("/content/drive/MyDrive/NLP/robot_news/ETNews(1).csv",index=False, encoding='utf-8-sig')